In [ ]:


# ------------------------------------------------------------------------------------
# Going through all image-gt pairs and collect all dice scores with gamma + otsu local
# ------------------------------------------------------------------------------------


import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path
from skimage.io import imread
from skimage import util
from skimage.filters import rank, threshold_otsu
from skimage.morphology import disk
from typing import Union, Tuple

# Defining a quicker otsu local for less computational load
def local_otsu_fast(image: np.ndarray, radius: int = 15) -> np.ndarray:
    """
    Efficient local Otsu threshold map.

    Args:
        image: 2D grayscale array (float or int).
        radius: local neighborhood radius.

    Returns:
        threshold_map: same shape and dtype as input, per-pixel Otsu thresholds.
    """
    # Must be uint8 for rank filters
    img_u8 = util.img_as_ubyte(image)

    # Define local neighborhood
    selem = disk(radius)

    # Compute local thresholds (C-optimized)
    local_thresh = rank.otsu(img_u8, selem)

    # Convert thresholds back to same dtype as input
    return local_thresh.astype(image.dtype)


# Current folder as project_root
# Going three layers of folders upwards to src
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root, "src")

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
from pre_processing import gammatransformation
from Complete_Otsu_Global import otsu_threshold_skimage_like
from Dice_Score import dice_score

# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, s) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)
    gt  = 1 - (gt == 0)         

    # Gamma transformation
    img_gamma = gammatransformation(img, gamma=0.6)
    
    # local Otsu tresholding
    t = local_otsu_fast(img_gamma, radius=s)

    # binarize
    binary = (img_gamma > t).astype(np.uint8)

    # 5. Compute Dice score
    return dice_score(binary.flatten(), gt.flatten())

# data set: NIH3T3
img_dir = Path(project_root) / "data-git" / "NIH3T3" / "img"
gt_dir = Path(project_root) / "data-git" / "NIH3T3" / "gt"

radius_dict = {
    0: 1100,
    1: 450,
    26: 290,
    27: 290,
    28: 310,
    29: 290,
    30: 270,
    31: 290,
    32: 170,
    33: 370,
    37: 350,
    40: 310,
    42: 250,
    44: 250,
    45: 230,
    46: 290,
    47: 650,
    49: 210
}

dice_scores_NIH3T3 = []

for img_file in sorted(img_dir.glob("dna-*.png")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('-')[-1]
    gt_file = gt_dir / f"{idx}.png"
    idx = int(idx)
    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue
    
    r = radius_dict.get(idx)
    if not isinstance(r, int):
     raise ValueError(f"Invalid radius for index {idx}: {r}")

    score = process_single(img_file, gt_file, s=r)
    dice_scores_NIH3T3.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_NIH3T3 = np.array(dice_scores_NIH3T3)           
print("\nDONE ⇢ Mean Dice:", dice_scores_NIH3T3.mean())

np.save('NIH3T3_dice_scores_gamma+local', dice_scores_NIH3T3)



# data set: N2DL-HeLa
img_dir = Path(project_root) / "data-git" / "N2DL-HeLa" / "img"
gt_dir = Path(project_root) / "data-git" / "N2DL-HeLa" / "gt"

radius_dict = {
    13: 270,
    52: 270,
    75: 70,
    79: 90
}

dice_scores_N2DL_HeLa = []

for img_file in sorted(img_dir.glob("t*.tif")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('t')[-1]
    gt_file = gt_dir / f"man_seg{idx}.tif"
    idx = int(idx)

    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue

    r = radius_dict.get(idx)

    score = process_single(img_file, gt_file, s=r)
    dice_scores_N2DL_HeLa.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_N2DL_HeLa = np.array(dice_scores_N2DL_HeLa)           
print("\nDONE ⇢ Mean Dice:", dice_scores_N2DL_HeLa.mean())

np.save('N2DL-HeLa_dice_scores_gamma+local', dice_scores_N2DL_HeLa)


# data set: N2DH-GOWT1
img_dir = Path(project_root) / "data-git" / "N2DH-GOWT1" / "img"
gt_dir = Path(project_root) / "data-git" / "N2DH-GOWT1" / "gt"

radius_dict = {
    72: 790,
    39: 790,
    1:  790,
    31: 790,
    21: 790,
    52: 770
}

dice_scores_N2DH_GOWT1 = []

for img_file in sorted(img_dir.glob("t*.tif")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('t')[-1]
    gt_file = gt_dir / f"man_seg{idx}.tif"
    idx = int(idx)

    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue
    
    r = radius_dict.get(idx)

    score = process_single(img_file, gt_file, s=r)
    dice_scores_N2DH_GOWT1.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_N2DH_GOWT1 = np.array(dice_scores_N2DH_GOWT1)           
print("\nDONE ⇢ Mean Dice:", dice_scores_N2DH_GOWT1.mean())

np.save('N2DH-GOWT1_dice_scores_gamma+local', dice_scores_N2DH_GOWT1)

dna-0.png            Dice = 0.9311
dna-1.png            Dice = 0.9337
dna-26.png           Dice = 0.8550
dna-27.png           Dice = 0.7973
dna-28.png           Dice = 0.8375
dna-29.png           Dice = 0.7448
dna-30.png           Dice = 0.6955
dna-31.png           Dice = 0.7661
dna-32.png           Dice = 0.5964
dna-33.png           Dice = 0.6761
dna-37.png           Dice = 0.6317
dna-40.png           Dice = 0.8117
dna-42.png           Dice = 0.7079
dna-44.png           Dice = 0.8567
dna-45.png           Dice = 0.8324
dna-46.png           Dice = 0.7008
dna-47.png           Dice = 0.6762
dna-49.png           Dice = 0.8638

DONE ⇢ Mean Dice: 0.7730358061082876
t13.tif              Dice = 0.7698
t52.tif              Dice = 0.8251
t75.tif              Dice = 0.8839
t79.tif              Dice = 0.8862

DONE ⇢ Mean Dice: 0.8412368017719345
t01.tif              Dice = 0.5524
t21.tif              Dice = 0.5760
t31.tif              Dice = 0.7839
t39.tif              Dice = 0.7582
t52.tif       